# Conditional Probability and Bayes' Theorem

## Introduction

Welcome to one of the most important concepts in probability and machine learning: **conditional probability**.

### What You'll Learn
1. What conditional probability means and how to calculate it
2. Bayes' theorem - the foundation of Bayesian machine learning
3. The difference between P(A|B) and P(B|A) - the base rate fallacy
4. Independence and how to test for it
5. Law of total probability
6. Real agricultural applications of these concepts

### Why This Matters
Conditional probability is everywhere in agriculture and ML:
- Disease diagnosis: "Given symptoms, what's the probability of disease?"
- Weather prediction: "Given current conditions, will it rain?"
- ML classification: "Given features, what's the probability of this class?"
- Bayesian methods: The entire field is built on conditional probability!

In [ ]:
# Import required libraries
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib_venn import venn2
import pandas as pd

# Set style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
np.random.seed(42)

print("✓ Libraries imported successfully!")
print("Ready to explore conditional probability!")

## 1. What is Conditional Probability?

**Conditional probability** is the probability of event A occurring, **given that** event B has already occurred.

**Notation**: P(A|B) reads as "probability of A given B"

### Formula
$$P(A|B) = \frac{P(A \cap B)}{P(B)}$$

provided P(B) > 0

### Intuition
When B has occurred, our sample space **shrinks** to just B. We then ask: "Of the outcomes in B, how many are also in A?"

In [ ]:
# Agricultural Example: Disease given Symptoms
# Survey of 1000 plants

total_plants = 1000

# Event D: Plant has disease
plants_with_disease = 150

# Event Y: Plant shows yellowing (symptom)
plants_with_yellowing = 300

# Both disease AND yellowing
plants_with_both = 120

# Calculate probabilities
P_D = plants_with_disease / total_plants
P_Y = plants_with_yellowing / total_plants
P_D_and_Y = plants_with_both / total_plants

# Conditional probability: P(Disease | Yellowing)
P_D_given_Y = P_D_and_Y / P_Y

print("Plant Disease and Symptoms Survey")
print("="*60)
print(f"Total plants surveyed: {total_plants}")
print(f"\nPlants with disease: {plants_with_disease} ({P_D:.1%})")
print(f"Plants with yellowing: {plants_with_yellowing} ({P_Y:.1%})")
print(f"Plants with BOTH: {plants_with_both} ({P_D_and_Y:.1%})")

print("\n" + "="*60)
print("\nConditional Probability Calculation:")
print(f"\nP(Disease | Yellowing) = P(Disease AND Yellowing) / P(Yellowing)")
print(f"                        = {P_D_and_Y:.2f} / {P_Y:.2f}")
print(f"                        = {P_D_given_Y:.2f}")

print("\n" + "="*60)
print(f"\n💡 Key Insight: If a plant shows yellowing, there's a {P_D_given_Y:.0%} chance")
print(f"   it has the disease. This is HIGHER than the base rate of {P_D:.0%}!")
print(f"   Yellowing is an informative symptom.")

In [ ]:
# Visualize conditional probability with Venn diagram
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))

# Standard probabilities
venn1 = venn2(subsets=(plants_with_disease - plants_with_both,
                       plants_with_yellowing - plants_with_both,
                       plants_with_both),
              set_labels=('Disease', 'Yellowing'),
              set_colors=('lightcoral', 'lightyellow'),
              alpha=0.7,
              ax=ax1)

if venn1.get_label_by_id('10'):
    venn1.get_label_by_id('10').set_text(f'{plants_with_disease - plants_with_both}\nDisease\nNo Yellow')
if venn1.get_label_by_id('01'):
    venn1.get_label_by_id('01').set_text(f'{plants_with_yellowing - plants_with_both}\nYellow\nNo Disease')
if venn1.get_label_by_id('11'):
    venn1.get_label_by_id('11').set_text(f'{plants_with_both}\nBoth')

ax1.set_title('All Plants\n(Full Sample Space)', fontsize=13, fontweight='bold')

# Conditional probability - focus on yellowing
venn2_plot = venn2(subsets=(0,  # Only disease (outside yellowing) - not shown
                            plants_with_yellowing - plants_with_both,
                            plants_with_both),
                   set_labels=('', 'Yellowing\n(New Sample Space)'),
                   set_colors=('white', 'lightyellow'),
                   alpha=0.9,
                   ax=ax2)

if venn2_plot.get_label_by_id('01'):
    venn2_plot.get_label_by_id('01').set_text(f'{plants_with_yellowing - plants_with_both}\nYellow Only')
if venn2_plot.get_label_by_id('11'):
    venn2_plot.get_label_by_id('11').set_text(f'{plants_with_both}\nDisease +\nYellow')

ax2.set_title(f'Given Yellowing: P(Disease|Yellow) = {plants_with_both}/{plants_with_yellowing} = {P_D_given_Y:.0%}',
              fontsize=13, fontweight='bold')

plt.tight_layout()
plt.show()

print("\n💡 Visual Insight: When we condition on yellowing, we ignore all plants")
print("   WITHOUT yellowing. The question becomes: 'Of yellow plants, how many have disease?'")

## 2. Bayes' Theorem: The Star of Probability

**Bayes' Theorem** lets us reverse conditional probabilities. If we know P(B|A), we can find P(A|B).

### The Formula
$$P(A|B) = \frac{P(B|A) \times P(A)}{P(B)}$$

### In words:
$$\text{Posterior} = \frac{\text{Likelihood} \times \text{Prior}}{\text{Evidence}}$$

Where:
- **P(A|B)**: Posterior - what we want to know
- **P(B|A)**: Likelihood - how likely is B if A is true
- **P(A)**: Prior - base rate of A
- **P(B)**: Evidence - total probability of B

### Why It Matters
Often we know P(symptoms | disease) but want P(disease | symptoms). Bayes' theorem makes this possible!

In [ ]:
# Classic Example: Disease Testing
# A test for crop disease

print("="*70)
print("BAYES' THEOREM: Disease Testing Example")
print("="*70)

# Given information
P_disease = 0.05  # 5% of fields have the disease (PRIOR)
P_positive_given_disease = 0.90  # Test is 90% sensitive (LIKELIHOOD)
P_positive_given_no_disease = 0.10  # 10% false positive rate

print("\nGiven Information:")
print("-"*70)
print(f"  P(Disease) = {P_disease:.0%} (prior - base rate)")
print(f"  P(Positive Test | Disease) = {P_positive_given_disease:.0%} (sensitivity)")
print(f"  P(Positive Test | No Disease) = {P_positive_given_no_disease:.0%} (false positive)")

# Calculate P(No Disease)
P_no_disease = 1 - P_disease

# Calculate P(Positive) using law of total probability
P_positive = (P_positive_given_disease * P_disease + 
              P_positive_given_no_disease * P_no_disease)

print("\nCalculate Total Probability of Positive Test (EVIDENCE):")
print("-"*70)
print(f"  P(Positive) = P(Pos|Disease)×P(Disease) + P(Pos|No Disease)×P(No Disease)")
print(f"              = {P_positive_given_disease:.2f}×{P_disease:.2f} + {P_positive_given_no_disease:.2f}×{P_no_disease:.2f}")
print(f"              = {P_positive:.4f}")

# Apply Bayes' Theorem
P_disease_given_positive = (P_positive_given_disease * P_disease) / P_positive

print("\nApply Bayes' Theorem (What we really want to know):")
print("-"*70)
print(f"  P(Disease | Positive Test) = P(Pos|Disease) × P(Disease) / P(Positive)")
print(f"                             = {P_positive_given_disease:.2f} × {P_disease:.2f} / {P_positive:.4f}")
print(f"                             = {P_disease_given_positive:.4f}")
print(f"                             = {P_disease_given_positive:.1%}")

print("\n" + "="*70)
print(f"\n🎯 RESULT: If a field tests positive, there's only a {P_disease_given_positive:.1%} chance")
print(f"   it actually has the disease!")
print(f"\n⚠️  SURPRISE: Even with a 90% accurate test, most positive results")
print(f"   are FALSE POSITIVES because the disease is rare (5% base rate).")
print("\n   This is why base rates (priors) matter so much!")
print("="*70)

In [ ]:
# Visualize Bayes' Theorem with a population of 1000 fields
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Simulate 1000 fields
n_fields = 1000
n_diseased = int(n_fields * P_disease)
n_healthy = n_fields - n_diseased

# Test results
n_diseased_positive = int(n_diseased * P_positive_given_disease)
n_diseased_negative = n_diseased - n_diseased_positive
n_healthy_positive = int(n_healthy * P_positive_given_no_disease)
n_healthy_negative = n_healthy - n_healthy_positive

# 1. Overall population
labels1 = ['Diseased', 'Healthy']
sizes1 = [n_diseased, n_healthy]
colors1 = ['lightcoral', 'lightgreen']
axes[0, 0].pie(sizes1, labels=labels1, colors=colors1, autopct='%1.0f%%',
              startangle=90, textprops={'fontsize': 11, 'fontweight': 'bold'})
axes[0, 0].set_title(f'Population of {n_fields} Fields\nBase Rate: {P_disease:.0%} diseased', 
                     fontsize=12, fontweight='bold')

# 2. Test results breakdown
categories = ['Diseased\n& Positive', 'Diseased\n& Negative', 
              'Healthy\n& Positive', 'Healthy\n& Negative']
values = [n_diseased_positive, n_diseased_negative, n_healthy_positive, n_healthy_negative]
colors2 = ['darkred', 'lightcoral', 'orange', 'lightgreen']

bars = axes[0, 1].bar(categories, values, color=colors2, edgecolor='black', linewidth=2)
axes[0, 1].set_ylabel('Number of Fields', fontsize=11)
axes[0, 1].set_title('Test Results Breakdown', fontsize=12, fontweight='bold')
axes[0, 1].tick_params(axis='x', rotation=45)

for i, (cat, val) in enumerate(zip(categories, values)):
    axes[0, 1].text(i, val + 5, str(val), ha='center', fontsize=10, fontweight='bold')

# 3. Among positive tests
n_total_positive = n_diseased_positive + n_healthy_positive
labels3 = [f'True Positive\n{n_diseased_positive}', f'False Positive\n{n_healthy_positive}']
sizes3 = [n_diseased_positive, n_healthy_positive]
colors3 = ['darkred', 'orange']
explode3 = (0.1, 0)

axes[1, 0].pie(sizes3, labels=labels3, colors=colors3, autopct='%1.1f%%',
              startangle=90, explode=explode3,
              textprops={'fontsize': 11, 'fontweight': 'bold'})
axes[1, 0].set_title(f'Among {n_total_positive} Positive Tests\nP(Disease|Positive) = {P_disease_given_positive:.1%}',
                     fontsize=12, fontweight='bold')

# 4. Summary table
axes[1, 1].axis('off')
summary_data = [
    ['', 'Diseased', 'Healthy', 'Total'],
    ['Positive', n_diseased_positive, n_healthy_positive, n_total_positive],
    ['Negative', n_diseased_negative, n_healthy_negative, n_diseased_negative + n_healthy_negative],
    ['Total', n_diseased, n_healthy, n_fields]
]

table = axes[1, 1].table(cellText=summary_data, cellLoc='center', loc='center',
                         colWidths=[0.2, 0.2, 0.2, 0.2])
table.auto_set_font_size(False)
table.set_fontsize(10)
table.scale(1, 2)

# Color header row and first column
for i in range(4):
    table[(0, i)].set_facecolor('lightgray')
    table[(0, i)].set_text_props(weight='bold')
    table[(i, 0)].set_facecolor('lightgray')
    table[(i, 0)].set_text_props(weight='bold')

# Color the key cell (true positives)
table[(1, 1)].set_facecolor('lightcoral')
table[(1, 2)].set_facecolor('lightyellow')

axes[1, 1].set_title('Confusion Matrix\n(Numbers out of 1000 fields)', 
                     fontsize=12, fontweight='bold', pad=20)

plt.tight_layout()
plt.show()

print(f"\n💡 Visual Insight: Out of {n_total_positive} positive tests, only {n_diseased_positive} are true positives!")
print(f"   The {n_healthy_positive} false positives outnumber the true positives.")

## 3. The Base Rate Fallacy

The **base rate fallacy** is ignoring the prior probability (base rate) when making judgments.

### Common Mistake
People think: "The test is 90% accurate, so a positive result means 90% chance of disease."

### Reality
Accuracy depends on:
1. Test sensitivity/specificity
2. **Base rate** (how common is the condition)

### Why P(A|B) ≠ P(B|A)
- P(symptoms | disease) is NOT the same as P(disease | symptoms)
- P(rain | clouds) is NOT the same as P(clouds | rain)

This confusion causes major errors in medical diagnosis, legal reasoning, and ML interpretation!

In [ ]:
# Demonstrate base rate impact
# Same test, different base rates

test_sensitivity = 0.90
test_false_positive = 0.10

base_rates = [0.01, 0.05, 0.10, 0.20, 0.50]
posteriors = []

print("Impact of Base Rate on P(Disease | Positive Test)")
print("="*60)
print(f"Test characteristics: {test_sensitivity:.0%} sensitive, {test_false_positive:.0%} false positive")
print("\n" + "-"*60)

for base_rate in base_rates:
    # Calculate using Bayes' theorem
    p_positive = (test_sensitivity * base_rate + 
                  test_false_positive * (1 - base_rate))
    p_disease_given_positive = (test_sensitivity * base_rate) / p_positive
    posteriors.append(p_disease_given_positive)
    
    print(f"Base rate {base_rate:5.0%} → P(Disease|Positive) = {p_disease_given_positive:5.1%}")

print("\n" + "="*60)
print("\n💡 Key Insight: Same test, dramatically different interpretations!")
print("   Base rate matters as much as test accuracy.")

In [ ]:
# Visualize base rate effect
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Line plot showing relationship
ax1.plot([br * 100 for br in base_rates], [p * 100 for p in posteriors], 
         'ro-', linewidth=3, markersize=10)
ax1.set_xlabel('Base Rate (% with disease)', fontsize=12)
ax1.set_ylabel('P(Disease | Positive Test) %', fontsize=12)
ax1.set_title('Impact of Base Rate on Diagnosis\n(Same test, different populations)', 
              fontsize=13, fontweight='bold')
ax1.grid(True, alpha=0.3)
ax1.axline((0, 0), slope=1, color='gray', linestyle='--', label='y=x (perfect correlation)', alpha=0.5)

# Add annotations
for br, post in zip(base_rates, posteriors):
    ax1.annotate(f'{post:.0%}', xy=(br*100, post*100), 
                xytext=(br*100, post*100 + 3),
                fontsize=9, fontweight='bold', ha='center')

# Bar chart comparison
scenarios = ['1%\nBase Rate', '5%\nBase Rate', '20%\nBase Rate']
selected_posteriors = [posteriors[0], posteriors[1], posteriors[3]]
colors = ['lightcoral', 'orange', 'darkred']

bars = ax2.bar(scenarios, [p * 100 for p in selected_posteriors], 
               color=colors, edgecolor='black', linewidth=2)
ax2.set_ylabel('P(Disease | Positive) %', fontsize=12)
ax2.set_title('Same 90% Accurate Test\nDifferent Base Rates', fontsize=13, fontweight='bold')
ax2.axhline(y=90, color='blue', linestyle='--', linewidth=2, 
            label='Test Accuracy (90%)', alpha=0.7)
ax2.legend()

for i, (scenario, post) in enumerate(zip(scenarios, selected_posteriors)):
    ax2.text(i, post * 100 + 2, f'{post:.1%}', ha='center', 
            fontsize=11, fontweight='bold')

plt.tight_layout()
plt.show()

## 4. Independence

Two events are **independent** if knowing one occurred doesn't change the probability of the other.

### Definitions (all equivalent):
A and B are independent if:
1. $$P(A|B) = P(A)$$
2. $$P(B|A) = P(B)$$
3. $$P(A \cap B) = P(A) \times P(B)$$

### Examples
- **Independent**: Coin flips, separate field conditions, unrelated weather events
- **Dependent**: Disease and symptoms, temperature and humidity, sequential crop stages

In [ ]:
# Test for independence

def test_independence(p_a, p_b, p_a_and_b, event_a_name="A", event_b_name="B"):
    """
    Test if two events are independent.
    """
    print(f"Testing Independence: {event_a_name} and {event_b_name}")
    print("="*60)
    print(f"P({event_a_name}) = {p_a:.3f}")
    print(f"P({event_b_name}) = {p_b:.3f}")
    print(f"P({event_a_name} AND {event_b_name}) = {p_a_and_b:.3f}")
    
    # Test: P(A∩B) = P(A) × P(B) ?
    expected_if_independent = p_a * p_b
    print(f"\nIf independent: P({event_a_name})×P({event_b_name}) = {expected_if_independent:.3f}")
    print(f"Actual: P({event_a_name} AND {event_b_name}) = {p_a_and_b:.3f}")
    
    difference = abs(p_a_and_b - expected_if_independent)
    print(f"\nDifference: {difference:.4f}")
    
    # Calculate conditional probabilities
    p_a_given_b = p_a_and_b / p_b
    p_b_given_a = p_a_and_b / p_a
    
    print(f"\nConditional Probabilities:")
    print(f"P({event_a_name}|{event_b_name}) = {p_a_given_b:.3f} vs P({event_a_name}) = {p_a:.3f}")
    print(f"P({event_b_name}|{event_a_name}) = {p_b_given_a:.3f} vs P({event_b_name}) = {p_b:.3f}")
    
    if difference < 0.01:  # Threshold for "close enough"
        print(f"\n✓ {event_a_name} and {event_b_name} are INDEPENDENT")
        print(f"  Knowing {event_b_name} doesn't affect probability of {event_a_name}")
        return True
    else:
        print(f"\n✗ {event_a_name} and {event_b_name} are DEPENDENT")
        print(f"  Knowing {event_b_name} changes probability of {event_a_name}")
        return False

# Example 1: Independent events (two separate fields)
P_frost_A = 0.15
P_frost_B = 0.15
P_both_frost = 0.0225  # = 0.15 × 0.15

test_independence(P_frost_A, P_frost_B, P_both_frost, 
                 "Frost in Field A", "Frost in Field B")

print("\n" + "="*60 + "\n")

# Example 2: Dependent events (disease and symptoms)
P_disease = 0.10
P_symptoms = 0.25
P_disease_and_symptoms = 0.08  # Much higher than 0.10 × 0.25 = 0.025

test_independence(P_disease, P_symptoms, P_disease_and_symptoms,
                 "Disease", "Symptoms")

## 5. Law of Total Probability

The **law of total probability** lets us calculate P(A) by breaking it into cases.

If B₁, B₂, ..., Bₙ partition the sample space (mutually exclusive and exhaustive):

$$P(A) = \sum_{i=1}^{n} P(A|B_i) \times P(B_i)$$

### Use Case
When A can happen through multiple pathways, calculate each pathway and add them up.

In [ ]:
# Agricultural Example: Overall crop yield
# Three soil types with different yield probabilities

print("Law of Total Probability: Overall High Yield Probability")
print("="*70)

# Soil type distribution
soil_types = ['Clay', 'Loam', 'Sandy']
P_soil = [0.30, 0.50, 0.20]  # Probability of each soil type

# Probability of high yield given each soil type
P_high_yield_given_soil = [0.60, 0.80, 0.50]

print("\nSoil Type Distribution and Yield Probabilities:")
print("-"*70)
for soil, p_soil, p_yield in zip(soil_types, P_soil, P_high_yield_given_soil):
    print(f"  P({soil:5s} soil) = {p_soil:.2f}  |  P(High Yield | {soil:5s}) = {p_yield:.2f}")

# Calculate overall probability using law of total probability
P_high_yield = sum(p_yield * p_soil 
                   for p_yield, p_soil in zip(P_high_yield_given_soil, P_soil))

print("\n" + "-"*70)
print("\nLaw of Total Probability:")
print(f"P(High Yield) = Σ P(High Yield | Soil_i) × P(Soil_i)")
print(f"              = ({P_high_yield_given_soil[0]:.2f}×{P_soil[0]:.2f}) + "
      f"({P_high_yield_given_soil[1]:.2f}×{P_soil[1]:.2f}) + "
      f"({P_high_yield_given_soil[2]:.2f}×{P_soil[2]:.2f})")

contributions = [p_yield * p_soil for p_yield, p_soil in zip(P_high_yield_given_soil, P_soil)]
print(f"              = {contributions[0]:.3f} + {contributions[1]:.3f} + {contributions[2]:.3f}")
print(f"              = {P_high_yield:.3f}")
print(f"              = {P_high_yield:.1%}")

print("\n" + "="*70)
print(f"\n💡 Result: Overall probability of high yield is {P_high_yield:.1%}")
print(f"   This is the weighted average across all soil types.")

In [ ]:
# Visualize law of total probability
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Stacked bar showing contributions
bottom = 0
for soil, contrib, color in zip(soil_types, contributions, ['brown', 'green', 'yellow']):
    ax1.bar('Total', contrib, bottom=bottom, label=f'{soil} contribution',
           color=color, edgecolor='black', linewidth=2, alpha=0.7)
    ax1.text(0, bottom + contrib/2, f'{contrib:.3f}\n({soil})', 
            ha='center', fontsize=10, fontweight='bold')
    bottom += contrib

ax1.set_ylabel('Probability', fontsize=12)
ax1.set_title('Law of Total Probability\nP(High Yield) = Sum of Contributions', 
              fontsize=13, fontweight='bold')
ax1.set_ylim([0, 0.8])
ax1.legend(loc='upper right')
ax1.text(0, P_high_yield + 0.02, f'Total = {P_high_yield:.3f}', 
        ha='center', fontsize=12, fontweight='bold')

# Tree diagram
ax2.axis('off')
ax2.set_xlim(0, 10)
ax2.set_ylim(0, 10)

# Draw tree
start_x, start_y = 1, 5
ax2.plot(start_x, start_y, 'ko', markersize=15)
ax2.text(start_x - 0.5, start_y, 'Start', fontsize=11, fontweight='bold', va='center')

# Branches to soil types
soil_y_positions = [8, 5, 2]
soil_colors = ['brown', 'green', 'yellow']

for soil, p_soil, p_yield, y_pos, color in zip(soil_types, P_soil, P_high_yield_given_soil, 
                                                 soil_y_positions, soil_colors):
    # Branch to soil type
    ax2.plot([start_x, 4], [start_y, y_pos], 'k-', linewidth=2, alpha=0.5)
    ax2.plot(4, y_pos, 'o', color=color, markersize=12, markeredgecolor='black', markeredgewidth=2)
    ax2.text(2.5, (start_y + y_pos)/2 + 0.2, f'{p_soil:.0%}', 
            fontsize=9, fontweight='bold', ha='center')
    ax2.text(4.5, y_pos, f'{soil}\nSoil', fontsize=9, fontweight='bold', va='center')
    
    # Branch to high yield
    ax2.plot([4, 7], [y_pos, y_pos], '-', color=color, linewidth=2)
    ax2.plot(7, y_pos, 'o', color='lightgreen', markersize=10, 
            markeredgecolor='black', markeredgewidth=2)
    ax2.text(5.5, y_pos + 0.3, f'{p_yield:.0%}', fontsize=8, fontweight='bold', ha='center')
    ax2.text(8, y_pos, f'High Yield\n{p_soil*p_yield:.3f}', 
            fontsize=8, fontweight='bold', va='center',
            bbox=dict(boxstyle='round', facecolor='lightgreen', alpha=0.7))

ax2.set_title('Tree Diagram: Multiple Pathways to High Yield', fontsize=13, fontweight='bold')

plt.tight_layout()
plt.show()

## 6. Comprehensive Agricultural Application

Let's combine all concepts in a realistic scenario: Crop Disease Management Decision

In [ ]:
# Scenario: Should we treat for disease based on symptoms?

print("="*70)
print("COMPREHENSIVE EXAMPLE: Disease Management Decision")
print("="*70)

# Given information
P_disease_early = 0.08  # Early season: 8% disease rate
P_disease_late = 0.20   # Late season: 20% disease rate
P_early_season = 0.60   # We're in early season 60% of growing period
P_late_season = 0.40

# Symptom probabilities
P_symptoms_given_disease = 0.85  # 85% of diseased plants show symptoms
P_symptoms_given_healthy = 0.15  # 15% of healthy plants show similar symptoms

print("\n1. BACKGROUND INFORMATION:")
print("-"*70)
print(f"Season stage: {P_early_season:.0%} early, {P_late_season:.0%} late")
print(f"Disease prevalence: {P_disease_early:.0%} (early), {P_disease_late:.0%} (late)")
print(f"Symptom rates: {P_symptoms_given_disease:.0%} if diseased, {P_symptoms_given_healthy:.0%} if healthy")

# Step 1: Overall disease probability (law of total probability)
P_disease_overall = (P_disease_early * P_early_season + 
                     P_disease_late * P_late_season)

print("\n2. OVERALL DISEASE PROBABILITY (Law of Total Probability):")
print("-"*70)
print(f"P(Disease) = P(Disease|Early)×P(Early) + P(Disease|Late)×P(Late)")
print(f"           = {P_disease_early:.2f}×{P_early_season:.2f} + {P_disease_late:.2f}×{P_late_season:.2f}")
print(f"           = {P_disease_overall:.3f}")

# Step 2: Overall probability of symptoms (law of total probability)
P_healthy = 1 - P_disease_overall
P_symptoms = (P_symptoms_given_disease * P_disease_overall + 
              P_symptoms_given_healthy * P_healthy)

print("\n3. OVERALL SYMPTOM PROBABILITY (Law of Total Probability):")
print("-"*70)
print(f"P(Symptoms) = P(Symptoms|Disease)×P(Disease) + P(Symptoms|Healthy)×P(Healthy)")
print(f"            = {P_symptoms_given_disease:.2f}×{P_disease_overall:.3f} + {P_symptoms_given_healthy:.2f}×{P_healthy:.3f}")
print(f"            = {P_symptoms:.4f}")

# Step 3: Bayes' theorem - probability of disease given symptoms
P_disease_given_symptoms = ((P_symptoms_given_disease * P_disease_overall) / 
                            P_symptoms)

print("\n4. DIAGNOSIS (Bayes' Theorem):")
print("-"*70)
print(f"P(Disease|Symptoms) = P(Symptoms|Disease) × P(Disease) / P(Symptoms)")
print(f"                    = {P_symptoms_given_disease:.2f} × {P_disease_overall:.3f} / {P_symptoms:.4f}")
print(f"                    = {P_disease_given_symptoms:.4f}")
print(f"                    = {P_disease_given_symptoms:.1%}")

# Decision analysis
treatment_cost = 50  # dollars per acre
crop_loss_if_diseased = 300  # dollars per acre if disease not treated

# Expected cost of treating
expected_cost_treat = treatment_cost

# Expected cost of not treating
expected_cost_no_treat = P_disease_given_symptoms * crop_loss_if_diseased

print("\n5. DECISION ANALYSIS:")
print("-"*70)
print(f"Treatment cost: ${treatment_cost}/acre")
print(f"Crop loss if diseased and not treated: ${crop_loss_if_diseased}/acre")
print(f"\nExpected cost if we TREAT: ${expected_cost_treat}/acre")
print(f"Expected cost if we DON'T TREAT: ${expected_cost_no_treat:.2f}/acre")

print("\n" + "="*70)
if expected_cost_treat < expected_cost_no_treat:
    savings = expected_cost_no_treat - expected_cost_treat
    print(f"\n✓ RECOMMENDATION: TREAT")
    print(f"  Expected savings: ${savings:.2f}/acre")
else:
    savings = expected_cost_treat - expected_cost_no_treat
    print(f"\n✓ RECOMMENDATION: DON'T TREAT")
    print(f"  Expected savings: ${savings:.2f}/acre")

print("\n  This decision accounts for:")
print("  • Base rate of disease (varies by season)")
print("  • Symptom reliability (not perfect indicator)")
print("  • Economic costs and benefits")
print("="*70)

## 7. Key Takeaways

### Conditional Probability
- P(A|B) = "probability of A given B happened"
- Formula: P(A|B) = P(A∩B) / P(B)
- Reduces sample space to just B

### Bayes' Theorem
- Lets us reverse conditional probabilities
- P(A|B) = P(B|A) × P(A) / P(B)
- Foundation of Bayesian statistics and ML

### Base Rate Fallacy
- Don't ignore prior probabilities!
- P(A|B) ≠ P(B|A) in general
- Test accuracy depends on base rate

### Independence
- P(A|B) = P(A) if independent
- P(A∩B) = P(A) × P(B) if independent
- Most real-world events are dependent

### Law of Total Probability
- Break complex probabilities into cases
- P(A) = Σ P(A|Bᵢ) × P(Bᵢ)
- Useful when A can happen multiple ways

### Connection to Machine Learning
- **Naive Bayes classifier**: Directly uses Bayes' theorem
- **Bayesian networks**: Encode conditional dependencies
- **Posterior probabilities**: Model outputs are conditional probabilities
- **Feature importance**: Tests for independence
- **Ensemble methods**: Combine conditional probabilities

### Common Mistakes to Avoid
- Confusing P(A|B) with P(B|A)
- Ignoring base rates
- Assuming independence without testing
- Forgetting to use law of total probability for P(B)

### Next Steps

You've completed the fundamentals of probability! Next:
1. Implement these concepts from scratch (Section 2)
2. Learn professional tools with SciPy (Section 3)
3. Apply to real agricultural problems (Section 4)

Continue to: `../2_from_scratch/probability_functions.py`

## Exercises (Optional)

Test your understanding:

1. **Conditional Probability**: 40% of fields are irrigated. Among irrigated fields, 80% have high yield. What's P(irrigated AND high yield)?

2. **Bayes' Theorem**: A disease affects 3% of plants. A test is 95% sensitive and has 10% false positive rate. If a plant tests positive, what's the probability it's actually diseased?

3. **Independence**: P(rain) = 0.3, P(cold) = 0.4, P(rain AND cold) = 0.15. Are these independent?

4. **Law of Total Probability**: Three varieties: A (30%), B (50%), C (20%). Success rates: A (70%), B (80%), C (60%). What's overall success probability?

Work in the cell below:

In [ ]:
# Your solutions here

# Exercise 1: Conditional Probability


# Exercise 2: Bayes' Theorem


# Exercise 3: Independence


# Exercise 4: Law of Total Probability

---

**Congratulations!** You've mastered conditional probability and Bayes' theorem!

These concepts are fundamental to:
- Statistical inference
- Machine learning algorithms
- Decision making under uncertainty
- Bayesian methods

**Next**: Implement these concepts from scratch in `../2_from_scratch/`